In [24]:
import pandas as pd
import datetime
import yfinance as yf

def _load_prices(
    ticker: str,
    data_source: str = 'yahoo',
    end: str = datetime.date.today()
) -> pd.DataFrame:
    return yf.download(ticker, start="2000-01-01", end=end, auto_adjust=False)['Adj Close']
    
def load_prices_multitickers(
    tickers: list,
    data_source: str = 'yahoo',
    end: str = datetime.date.today()
) -> list:
    prices = pd.DataFrame()
    for t in tickers:
        prices[t] = _load_prices(t, data_source, end)
        
    return prices.fillna(value=None, method="ffill", axis=0, inplace=False).dropna(axis=0, how='any', inplace=False)

In [38]:
import pandas as pd
from pypfopt import expected_returns
from pypfopt import risk_models

def _calc_returns(
    prices: pd.DataFrame,
    risk_free_rate: float = 0.04
) -> dict:
    rets = {}
    rets['mhr'] = expected_returns.mean_historical_return(prices, frequency=252)
    rets['ehr'] = expected_returns.ema_historical_return(prices, frequency=252, span=252*5)
    rets['capmr'] = expected_returns.capm_return(prices,  risk_free_rate=risk_free_rate, compounding=True, frequency=252)

    return rets

def _calc_riskcovs(
    prices: pd.DataFrame,
    risk_free_rate: float = 0.04
) -> dict:
    covs = {}
    covs['sample_cov'] = risk_models.risk_matrix(prices, method='sample_cov')
    covs['semicovariance'] = risk_models.risk_matrix(prices, method='semicovariance', benchmark=risk_free_rate, frequency=252, log_returns=False)
    covs['exp_cov'] = risk_models.risk_matrix(prices, method='exp_cov', span=252*5, frequency=252)

    covs['ledoiid_wolf_cc'] = risk_models.risk_matrix(prices, method='ledoit_wolf_constant_correlation', frequency=252)
    covs['ledoiid_wolf_sf'] = risk_models.risk_matrix(prices, method='ledoit_wolf_single_factor', frequency=252)
    covs['ledoiid_wolf_cv'] = risk_models.risk_matrix(prices, method='ledoit_wolf_constant_variance', frequency=252)
    covs['oracle_approx'] = risk_models.risk_matrix(prices, method='oracle_approximating', frequency=252)
    
    return covs

def calc_returns_riskcovs(
    prices: pd.DataFrame,
    risk_free_rate: float = 0.04
) -> (dict, dict):
    r = _calc_returns(prices)
    c = _calc_riskcovs(prices)
    return r, c

In [39]:
# 各指数に連動するETFのティッカーシンボル
etf_tickers = {
    "S&P 500": "SPY",  # SPDR S&P 500 ETF Trust
    "Russell 1000": "IWB",  # iShares Russell 1000 ETF
    "FTSE Global All Cap ex US Index": "VFWAX",  # Vanguard FTSE All-World ex-US Index Fund
    "MSCI ACWI ex-US Index": "ACWX",  # iShares MSCI ACWI ex U.S. ETF
    "Bloomberg US Aggregate Bond Index": "AGG",  # iShares Core U.S. Aggregate Bond ETF
    "FTSE World Government Bond Index (WGBI)": "BWX",  # SPDR Bloomberg Barclays International Treasury Bond ETF
    "Bloomberg Global Aggregate Bond Index": "BNDX"  # Vanguard Total International Bond ETF
}
prices = load_prices_multitickers(etf_tickers.values())
calc_returns_riskcovs(prices)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_76/3354200728.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return prices.fillna(value=None, method="ffill", axis=0, inplace=False).dropna(axis=0, how='any', inplace=False)


({'mhr': SPY      0.137475
  IWB      0.135214
  VFWAX    0.047965
  ACWX     0.045742
  AGG      0.015875
  BWX     -0.014649
  BNDX     0.023484
  dtype: float64,
  'ehr': SPY      0.182484
  IWB      0.179482
  VFWAX    0.058165
  ACWX     0.063057
  AGG      0.008905
  BWX     -0.029434
  BNDX     0.021062
  Name: 2024-12-30 00:00:00, dtype: float64,
  'capmr': SPY      0.070029
  IWB      0.070519
  VFWAX    0.068238
  ACWX     0.070683
  AGG      0.042956
  BWX      0.046317
  BNDX     0.042043
  Name: mkt, dtype: float64},
 {'sample_cov':             SPY       IWB     VFWAX      ACWX       AGG       BWX      BNDX
  SPY    0.028497  0.028868  0.022437  0.024456  0.000738  0.001648  0.000628
  IWB    0.028868  0.029376  0.022821  0.024863  0.000760  0.001673  0.000662
  VFWAX  0.022437  0.022821  0.024343  0.026046  0.001025  0.003640  0.000563
  ACWX   0.024456  0.024863  0.026046  0.028504  0.001136  0.003955  0.000650
  AGG    0.000738  0.000760  0.001025  0.001136  0.002587  0

In [22]:
import yfinance as yf
import pandas as pd

# 各指数に連動するETFのティッカーシンボル
etf_tickers = {
    "S&P 500": "^GSPC",  # SPDR S&P 500 ETF Trust
    "Russell 1000": "IWB",  # iShares Russell 1000 ETF
    "FTSE Global All Cap ex US Index": "VFWAX",  # Vanguard FTSE All-World ex-US Index Fund
    "MSCI ACWI ex-US Index": "ACWX",  # iShares MSCI ACWI ex U.S. ETF
    "Bloomberg US Aggregate Bond Index": "AGG",  # iShares Core U.S. Aggregate Bond ETF
    "FTSE World Government Bond Index (WGBI)": "BWX",  # SPDR Bloomberg Barclays International Treasury Bond ETF
    "Bloomberg Global Aggregate Bond Index": "BNDX"  # Vanguard Total International Bond ETF
}

# データを取得
data = {}
for index, ticker in etf_tickers.items():
    data[index] = yf.download(ticker, start="2000-01-01", end="2023-01-01", auto_adjust=False)['Adj Close']

# データを表示
for index, df in data.items():
    print(f"{index} (ETF: {etf_tickers[index]}) - Adj Close:")
    print(df.head())
    print("\n")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

S&P 500 (ETF: ^GSPC) - Adj Close:
Ticker            ^GSPC
Date                   
2000-01-03  1455.219971
2000-01-04  1399.420044
2000-01-05  1402.109985
2000-01-06  1403.449951
2000-01-07  1441.469971


Russell 1000 (ETF: IWB) - Adj Close:
Ticker            IWB
Date                 
2000-05-19  48.677158
2000-05-22  48.533882
2000-05-23  47.715099
2000-05-24  48.124496
2000-05-25  47.766266


FTSE Global All Cap ex US Index (ETF: VFWAX) - Adj Close:
Ticker          VFWAX
Date                 
2011-09-27  16.635542
2011-09-28  16.356070
2011-09-29  16.608919
2011-09-30  16.169741
2011-10-03  15.683990


MSCI ACWI ex-US Index (ETF: ACWX) - Adj Close:
Ticker           ACWX
Date                 
2008-04-01  33.702732
2008-04-02  34.030827
2008-04-03  34.332684
2008-04-04  34.417984
2008-04-07  34.588604


Bloomberg US Aggregate Bond Index (ETF: AGG) - Adj Close:
Ticker            AGG
Date                 
2003-09-29  52.085358
2003-09-30  52.355530
2003-10-01  52.330067
2003-10-02  52.248